<a href="https://colab.research.google.com/github/gmrdns03/DataAnalysis_Python_Minkyo/blob/main/%5B%ED%95%B8%EC%A6%88%EC%98%A8%5D%20%EB%84%A4%EC%9D%B4%EB%B2%84%EC%98%81%ED%99%94%20TOP100_%ED%81%AC%EB%A1%A4%EB%A7%81%20%26%20%EC%98%81%ED%99%94%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_TOP100%ED%81%AC%EB%A1%A4%EB%A7%81_%26%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**URL**=
Uniform Resource Locator : <br>
인터넷에 접속할 때, 네트워크 상에서 해당 자원이 어디 있는지 알려주는 문자열(사이트의 집주소)이다.

# 네이버 영화 TOP100크롤링 하기

In [ ]:
# 라이브러리 설치
!pip install selenium

     |████████████████████████████████| 911kB 8.0MB/s 


In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804

# 분석할 데이터 모으기(크롤링)

## 네이버 영화 TOP100 페이지 별 링크 수집

In [ ]:
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_R&page= 실시간(1, 5)
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_D&page= 일간(1, 5)
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_W&page= 주간(1, 5)
# https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page= 월간(1, 5)

def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = []  # 100개의 영화 링크를 담을 리스트 변수 선언

  for pageNo in range(startPageNo -1, lastPageNo) :
    # 실시간 TOP 100 영화 링크 수집
    url = "https://serieson.naver.com/movie/top100List.nhn?&rankingTypeCode=PC_M&page=" + str(pageNo+1)

    req = requests.get(url)  # 링크에 접속하는 함수/ 접속할 수 있는 정보가 req에 담긴다.

    # 페이지의 소스를 긁어와서 저장한다. "<a" 테그에 해당 페이지로 접속할 수 있는 링크가 담겨있다.
    soup = BeautifulSoup(req.text, 'lxml') # html 파싱을 위한 객체 생성

    # <div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href=""> 가져오기
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]')  # ul안에 li안에 a안에 href의 내용을가져오라

    for movielink in movielinks :
      link = str(movielink.get('href'))
      # 접속할 수 있는전체 링크 형태로 변환해서 append
      links.append("https://series.naver.com"+link)  

  return links



In [ ]:
links = getPageLinksWantRange(1, 5)
print(links)

['https://series.naver.com/movie/detail.nhn?productNo=6291892', 'https://series.naver.com/movie/detail.nhn?productNo=3400212', 'https://series.naver.com/movie/detail.nhn?productNo=3024534', 'https://series.naver.com/movie/detail.nhn?productNo=2822635', 'https://series.naver.com/movie/detail.nhn?productNo=6315371', 'https://series.naver.com/movie/detail.nhn?productNo=3639460', 'https://series.naver.com/movie/detail.nhn?productNo=6023769', 'https://series.naver.com/movie/detail.nhn?productNo=6283368', 'https://series.naver.com/movie/detail.nhn?productNo=6243813', 'https://series.naver.com/movie/detail.nhn?productNo=6119777', 'https://series.naver.com/movie/detail.nhn?productNo=6139375', 'https://series.naver.com/movie/detail.nhn?productNo=2686058', 'https://series.naver.com/movie/detail.nhn?productNo=2747742', 'https://series.naver.com/movie/detail.nhn?productNo=3293948', 'https://series.naver.com/movie/detail.nhn?productNo=6176165', 'https://series.naver.com/movie/detail.nhn?productNo=6

## 네이버 영화 제복, 평점, 장르, 줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links) :
  title_infos = []  # 제목
  content_infos = []  # 줄거리
  genre_infos = []  # 장르
  score_infos = []  # 평점
  date_infos = []  # 개봉일

  url2 = "https://www.naver.com"  # 네이버에서 중간에 트래픽으로 인해 IP가 막히는 것을 막기 위해 우회

  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(url2)
  time.sleep(3.0)  # 중간에 3초 정도 쉬게 만든다.

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't')

  for link in links :
    print(link + '  수 집 중........')
    driver.switch_to.window(driver.window_handles[-1])  # url을 넣기 위해 새탭 열고 주소창을 활성화 하는 작업
    time.sleep(0.1)
    driver.get(link)  # 접속한다.
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0])  # 새창 활성화
    time.sleep(3.0)

    html_source = driver.page_source  # BeautifulSoup(req.text, 'lxml') 같은 과정
    html_soup = BeautifulSoup(html_source, 'lxml')

    # 청소년 관람 불가 영화 크롤링의 경우 -> 인증 -> 수집 제외하기
    flag = html_soup.text[0:10]

    newflag = "".join(flag)
    newflag = newflag.replace('\n', '')

    if newflag == '네이버' :
      time.sleep(1.0)

      #평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em ')
      score = float(score.text)
      score = int(score) # 정수형으로 변환
      score_infos.append(score)
      print(score)

      #장르 수집
      # soup의 함수를 쓸 때에는 공백을 꺽쇠로 인식한다.
      # 하지만 driver의 css가 추출할 때에는 >를 써서 추적 한다.
      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
      genre = genre.replace('장르', '')
      genre = genre.replace('\n', '')  # 크롤링할 때 쓰레기 정보들을 지우고 수집하지 않으면 나중에 전처리하기 힘들어진다.
      genre = genre.replace('\t', '')
      genre_infos.append(genre)

      # 제목, 줄거리 수집 (줄거리는 영화 상세정보 안에 있었다.)
      # 그 자리에 있으면 추출 없으면 예외사항 발생 -> try, except를 사용

      try:
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href')
      
      except:  # 오류가 안나서 편함
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href')

      # 영화 상세정보 보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)  # 해당 페이지로 이동

      # 제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find("meta", {"property":"og:title"}).get('content')
      title_infos.append(title)
      print(title)

      # 줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx')

      if len(contents_texts) == 0:
        content_infos.append("줄거리 오류")

      else :
        for contents in contents_texts :
          # 줄거리 데이터 클렌징 작업
          temp = contents.text
          temp = temp.replace('\r', '')  # 한줄 개행
          temp = temp.replace('\xa0', '')  # 공백 제거
          content_infos.append(temp)

    elif newflag == '' :
      print('청불 영화로 데이터 수집하지 않습니다.')

  print('수집 완료합니다....')
  print(len(score_infos), len(genre_infos), len(content_infos))
  driver.close()

  movie_dic = {"제목": title_infos, "평점": score_infos, "장르": genre_infos, "줄거리": content_infos}

  movie_df = pd.DataFrame(movie_dic)

  # 수집된 정보 중에 중복 데이터 삭제
  # 줄거리 비교해서 중복된 영화를 삭제(첫번째 수집 영화만 남음)
  movie_df2 = movie_df.drop_duplicates("줄거리", keep='first')  

  return movie_df2


# CSV파일로 저장

In [ ]:
def dftoCsv(movie_df, num) :
  try:
    movie_df.to_csv(('movie_my_data' + str(num) +'.csv'), 
                    sep=',', na_rep='NaN', encoding='euc-kr')
    
  except:
    print("Error")

  

# 크롤링 실행

In [ ]:
# 1) 크롤링 할 링크 수집
links = getPageLinksWantRange(1, 5)

# 2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

# 3) csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

https://series.naver.com/movie/detail.nhn?productNo=6119777  수 집 중........
7
미나리
https://series.naver.com/movie/detail.nhn?productNo=6176165  수 집 중........
8
노바디 
https://series.naver.com/movie/detail.nhn?productNo=6158879  수 집 중........
9
자산어보
https://series.naver.com/movie/detail.nhn?productNo=6098871  수 집 중........
9
라야와 마지막 드래곤
https://series.naver.com/movie/detail.nhn?productNo=6023769  수 집 중........
9
소울
https://series.naver.com/movie/detail.nhn?productNo=6139377  수 집 중........
8
고질라 VS. 콩
https://series.naver.com/movie/detail.nhn?productNo=6139375  수 집 중........
9
소울
https://series.naver.com/movie/detail.nhn?productNo=6119778  수 집 중........
7
미나리
https://series.naver.com/movie/detail.nhn?productNo=4356726  수 집 중........
9
어벤져스: 엔드게임
https://series.naver.com/movie/detail.nhn?productNo=6176518  수 집 중........
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=3293948  수 집 중........
청불 영화로 데이터 수집하지 않습니다.
https://series.naver.com/movie/detail.nhn?productNo=6039